In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
import joblib
from datetime import datetime

# 1. Load and prepare data
weather_data = pd.read_csv("/content/Weather.csv")
solar_data = pd.read_excel("/content/Solar work shop.xlsx")

# Convert dates
solar_data['Date'] = pd.to_datetime(solar_data['Date'], errors='coerce')
weather_data['datetime'] = pd.to_datetime(weather_data['datetime'], dayfirst=True, errors='coerce')

# 2. Merge datasets
data = pd.merge(weather_data, solar_data, left_on='datetime', right_on='Date', how='inner')

# 3. Select features and target
features = ['temp', 'humidity', 'dew', 'precip', 'cloudcover', 'solarradiation', 'solarenergy']
target = 'KWH'
data = data.dropna(subset=features + [target])

# 4. Scale features and target
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X = scaler_X.fit_transform(data[features])
y = scaler_y.fit_transform(data[[target]])

# 5. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Train SVR model
model = SVR()
model.fit(X_train, y_train.ravel())

# 7. Evaluate model
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
y_true = scaler_y.inverse_transform(y_test.reshape(-1, 1))

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)
mean_actual = np.mean(y_true)
accuracy = 1 - (rmse / mean_actual)

print(f"SVR RMSE: {rmse:.2f}")
print(f"SVR R² Score: {r2:.2f}")
print(f"SVR Accuracy (approx.): {accuracy * 100:.2f}%")

# 8. Save model PROPERLY with all required components
model_data = {
    'model': model,
    'scaler_X': scaler_X,
    'scaler_y': scaler_y,
    'features': features,
    'target': target,
    'metadata': {
        'training_date': datetime.now().isoformat(),
        'model_type': 'SVR',
        'version': '1.0'
    }
}

# Save with compression for smaller file size
joblib.dump(model_data, 'solar_svr_model.joblib', compress=3, protocol=4)

# 9. Verify the saved model can be loaded
def load_model(path):
    try:
        loaded = joblib.load(path)
        print("Model loaded successfully!")
        print(f"Model expects {loaded['model'].n_features_in_} features")
        return loaded
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

# Test loading
loaded_model = load_model('solar_svr_model.joblib')

# 10. Prediction function using loaded model
def predict_kwh(model_data, input_data):
    try:
        # Convert input to DataFrame
        input_df = pd.DataFrame([input_data])[model_data['features']]

        # Scale features
        input_scaled = model_data['scaler_X'].transform(input_df)

        # Predict
        pred_scaled = model_data['model'].predict(input_scaled)

        # Unscale prediction
        return model_data['scaler_y'].inverse_transform(pred_scaled.reshape(-1, 1))[0][0]
    except Exception as e:
        print(f"Prediction error: {e}")
        return None

# Example prediction
if loaded_model:
    test_input = {
        'temp': 32.4,
        'humidity': 43.4,
        'dew': 17.3,
        'precip': 0,
        'cloudcover': 40,
        'solarradiation': 289.3,
        'solarenergy': 25.0
    }

    prediction = predict_kwh(loaded_model, test_input)
    print(f"Predicted Solar Energy (KWH): {prediction:.2f}")

# 11. Download the model file (for Colab)
from google.colab import files
files.download('solar_svr_model.joblib')

SVR RMSE: 123.40
SVR R² Score: 0.35
SVR Accuracy (approx.): 61.30%
Model loaded successfully!
Model expects 7 features
Predicted Solar Energy (KWH): 408.92


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>